In [ ]:
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

Saved /Users/sonalimanoharan/Desktop/scientific_research/hw/figures/class_distribution_per_subject.png and /Users/sonalimanoharan/Desktop/scientific_research/hw/figures/class_distribution_per_subject.pdf


In [ ]:
try:
    script_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    script_dir = os.path.join(os.getcwd(), "figures")
    if not os.path.exists(os.path.join(script_dir, "windows_per_subject.csv")):
        script_dir = os.getcwd()
csv_path = os.path.join(script_dir, "windows_per_subject.csv")
df = pd.read_csv(csv_path)
df = df.sort_values("Hand-washing", ascending=True)  

fig, ax = plt.subplots(figsize=(10, 6))
subject_labels = [s[:20] + "..." if len(s) > 20 else s for s in df["Subject ID"]]
x = np.arange(len(df))
width = 0.35

bars1 = ax.bar(x - width/2, df["Non-hand-washing"], width, label="Non-hand-washing (class 0)", color="#4A90D9", edgecolor="black", linewidth=0.5)
bars2 = ax.bar(x + width/2, df["Hand-washing"], width, label="Hand-washing (class 1)", color="#F0AD4E", edgecolor="black", linewidth=0.5)

In [ ]:
ax.set_ylabel("Number of windows", fontsize=11)
ax.set_xlabel("Subject ID", fontsize=11)
ax.set_title("Window Distribution per Subject (TS2Vec pipeline: 500 samples, 50% overlap)", fontsize=12, fontweight="bold")
ax.set_xticks(x)
ax.set_xticklabels(subject_labels, rotation=45, ha="right", fontsize=8)
ax.legend(loc="upper right", fontsize=10)
ax.grid(axis="y", alpha=0.3)

total_hw = df["Hand-washing"].sum()
total_non = df["Non-hand-washing"].sum()
ratio = 100 * total_hw / (total_hw + total_non)
ax.text(0.02, 0.98, f"Overall hand-washing ratio: {ratio:.2f}%", transform=ax.transAxes,
        fontsize=10, verticalalignment="top", bbox=dict(boxstyle="round", facecolor="wheat", alpha=0.5))

plt.tight_layout()
out_png = os.path.join(script_dir, "class_distribution_per_subject.png")
out_pdf = os.path.join(script_dir, "class_distribution_per_subject.pdf")
fig.savefig(out_png, dpi=150, bbox_inches="tight", facecolor="white")
fig.savefig(out_pdf, bbox_inches="tight", facecolor="white")
print(f"Saved {out_png} and {out_pdf}")
plt.close()


In [ ]:
try:
    BASE = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
except NameError:
    BASE = os.getcwd()  # Run notebook from project root (hw/)
FIG_DIR = os.path.join(BASE, "save_model_mlp_xgb")
PATHS = {
    "compare": os.path.join(BASE, "fair_model", "compare_fair_f1_results.csv"),
    "compare_swapped": os.path.join(BASE, "fair_model_swapped_labels", "compare_swapped_labels_results.csv"),
    "ts2vec_humidity": os.path.join(BASE, "ts2vec_saved_model", "TS2Vec_Humidity_frozen.csv"),
    "ts2vec_nohumidity": os.path.join(BASE, "ts2vec_no_humid_saved_model", "TS2Vec_IMU_Only_frozen.csv"),
}

MODEL_NAMES = {
    "compare": "CNN-LSTM\n(Supervised)",
    "compare_swapped": "CNN-LSTM\n(Swapped labels)",
    "ts2vec_humidity": "TS2Vec+Humidity\n(Self-supervised)",
    "ts2vec_nohumidity": "TS2Vec IMU-only\n(Self-supervised)",
}

COLORS = {
    "compare": "#2E86AB",           # blue
    "compare_swapped": "#A23B72",   # magenta (sanity check)
    "ts2vec_humidity": "#28A745",   # green
    "ts2vec_nohumidity": "#F0AD4E",  # orange
}


In [ ]:
def load_data():
    data = {}
    for key, path in PATHS.items():
        if os.path.exists(path):
            data[key] = pd.read_csv(path)
        else:
            print(f"Warning: {path} not found")
    return data

In [ ]:
def get_metrics(data):
    metrics = []
    for key, df in data.items():
        if key in ["compare", "compare_swapped"]:
            f1_fixed = df["fair_f1_fixed"].mean()
            f1_val = df["fair_f1_val_threshold"].mean()
            acc_fixed = df["fair_acc_fixed"].mean()
            acc_val = df["fair_acc_val_threshold"].mean()
        else:
            f1_fixed = f1_val = df["f1_score"].mean()
            acc_fixed = acc_val = df["accuracy"].mean()
        metrics.append({
            "model": key,
            "name": MODEL_NAMES[key],
            "f1": f1_fixed,
            "accuracy": acc_fixed,
            "f1_fixed": f1_fixed,
            "f1_val_threshold": f1_val,
            "acc_fixed": acc_fixed,
            "acc_val_threshold": acc_val,
            "n_subjects": len(df),
        })
    return pd.DataFrame(metrics)

In [ ]:
def fig1_model_comparison_bar(metrics_df, out_dir):
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    
    models = metrics_df["name"].tolist()
    x = np.arange(len(models))
    width = 0.35
    
    ax1 = axes[0]
    bars = ax1.bar(x - width/2, metrics_df["f1"], width, color=[COLORS.get(m, "#666") for m in metrics_df["model"]],edgecolor="black", linewidth=0.8)
    ax1.set_ylabel("Mean F1 Score", fontsize=11)
    ax1.set_title("(a) F1 Score (Hand-washing Detection)", fontsize=12, fontweight="bold")
    ax1.set_xticks(x)
    ax1.set_xticklabels(models, fontsize=9, wrap=True)
    ax1.set_ylim(0, 1.05)
    ax1.axhline(y=0.5, color="gray", linestyle="--", alpha=0.5)
    ax1.grid(axis="y", alpha=0.3)
    for i, (bar, v) in enumerate(zip(bars, metrics_df["f1"])):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, f"{v:.3f}", ha="center", va="bottom", fontsize=9, fontweight="bold")

    ax2 = axes[1]
    bars = ax2.bar(x - width/2, metrics_df["accuracy"], width, color=[COLORS.get(m, "#666") for m in metrics_df["model"]], edgecolor="black", linewidth=0.8)
    ax2.set_ylabel("Mean Accuracy", fontsize=11)
    ax2.set_title("(b) Accuracy", fontsize=12, fontweight="bold")
    ax2.set_xticks(x)
    ax2.set_xticklabels(models, fontsize=9, wrap=True)
    ax2.set_ylim(0.8, 1.02)
    ax2.grid(axis="y", alpha=0.3)
    for i, (bar, v) in enumerate(zip(bars, metrics_df["accuracy"])):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005, f"{v:.3f}",ha="center", va="bottom", fontsize=9, fontweight="bold")
    
    plt.suptitle("Model Comparison: Supervised vs Self-Supervised Hand Washing Detection", fontsize=13, fontweight="bold", y=1.02)
    plt.tight_layout()
    fig.savefig(os.path.join(out_dir, "ml_model_comparison.png"), dpi=150, bbox_inches="tight", facecolor="white")
    fig.savefig(os.path.join(out_dir, "ml_model_comparison.pdf"), bbox_inches="tight", facecolor="white")
    plt.close()
    print("Saved ml_model_comparison.png/pdf")

In [ ]:
def fig2_f1_boxplot(data, out_dir):
    excluded = {"2025-01-18-22-38-29_37959204-490b-4cd9-b647-94e743071951", 
                "2025-01-28-21-43-21_e4380fee-3c78-4e38-936f-acd60513e279"}
    
    plot_data = []
    labels = []
    colors_list = []
    
    for key in ["compare", "ts2vec_humidity", "ts2vec_nohumidity"]:
        if key not in data:
            continue
        df = data[key]
        if key.startswith("compare"):
            f1_col = "fair_f1_fixed"
            df_plot = df[~df["subject"].isin(excluded)] if "subject" in df.columns else df
        else:
            f1_col = "f1_score"
            df_plot = df
        vals = df_plot[f1_col].values
        plot_data.append(vals)
        labels.append(MODEL_NAMES[key].replace("\n", " "))
        colors_list.append(COLORS[key])
    
    fig, ax = plt.subplots(figsize=(8, 5))
    bp = ax.boxplot(plot_data, tick_labels=labels, patch_artist=True, showmeans=True)
    for patch, c in zip(bp["boxes"], colors_list):
        patch.set_facecolor(c)
        patch.set_alpha(0.7)
    ax.set_ylabel("F1 Score (per subject)", fontsize=11)
    ax.set_title("F1 Score Distribution Across Subjects (18 subjects, excluding swapped-labels)", fontsize=12, fontweight="bold")
    ax.set_ylim(0, 1.05)
    ax.grid(axis="y", alpha=0.3)
    plt.tight_layout()
    fig.savefig(os.path.join(out_dir, "ml_f1_boxplot.png"), dpi=150, bbox_inches="tight", facecolor="white")
    fig.savefig(os.path.join(out_dir, "ml_f1_boxplot.pdf"), bbox_inches="tight", facecolor="white")
    plt.close()
    print("Saved ml_f1_boxplot.png/pdf")

In [ ]:
def fig3_humidity_ablation(data, out_dir):
    if "ts2vec_humidity" not in data or "ts2vec_nohumidity" not in data:
        print("Skipping humidity ablation: missing TS2Vec data")
        return
    
    df_h = data["ts2vec_humidity"].merge(
        data["ts2vec_nohumidity"], on="subject", suffixes=("_hum", "_nohum")
    )
    df_h["f1_diff"] = df_h["f1_score_hum"] - df_h["f1_score_nohum"]
    df_h = df_h.sort_values("f1_diff", ascending=True)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    x = np.arange(len(df_h))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, df_h["f1_score_nohum"], width, label="TS2Vec IMU-only", color=COLORS["ts2vec_nohumidity"], edgecolor="black")
    bars2 = ax.bar(x + width/2, df_h["f1_score_hum"], width, label="TS2Vec + Humidity", color=COLORS["ts2vec_humidity"], edgecolor="black")
    
    subj_labels = [s[:18] + "..." if len(s) > 18 else s for s in df_h["subject"]]
    ax.set_xticks(x)
    ax.set_xticklabels(subj_labels, rotation=45, ha="right", fontsize=8)
    ax.set_ylabel("F1 Score", fontsize=11)
    ax.set_title("Effect of Humidity: TS2Vec IMU-only vs TS2Vec + Humidity (per subject)", fontsize=12, fontweight="bold")
    ax.legend(loc="upper right")
    ax.set_ylim(0, 1.05)
    ax.grid(axis="y", alpha=0.3)
    
    mean_diff = df_h["f1_diff"].mean()
    ax.axhline(y=df_h["f1_score_hum"].mean(), color=COLORS["ts2vec_humidity"], linestyle="--", alpha=0.5)
    ax.axhline(y=df_h["f1_score_nohum"].mean(), color=COLORS["ts2vec_nohumidity"], linestyle="--", alpha=0.5)
    ax.text(0.98, 0.02, f"Mean F1 gain with humidity: +{mean_diff:.3f}", transform=ax.transAxes, ha="right", va="bottom", fontsize=10, bbox=dict(boxstyle="round", facecolor="wheat", alpha=0.7))
    
    plt.tight_layout()
    fig.savefig(os.path.join(out_dir, "ml_humidity_ablation.png"), dpi=150, bbox_inches="tight", facecolor="white")
    fig.savefig(os.path.join(out_dir, "ml_humidity_ablation.pdf"), bbox_inches="tight", facecolor="white")
    plt.close()
    print("Saved ml_humidity_ablation.png/pdf")

In [ ]:
def fig4_supervised_vs_selfsupervised(data, out_dir):
    excluded = {"2025-01-18-22-38-29_37959204-490b-4cd9-b647-94e743071951",
                "2025-01-28-21-43-21_e4380fee-3c78-4e38-936f-acd60513e279"}
    
    if "compare" not in data or "ts2vec_humidity" not in data:
        print("Skipping supervised vs self-supervised: missing data")
        return
    
    df_c = data["compare"][~data["compare"]["subject"].isin(excluded)][["subject", "fair_f1_fixed"]].rename(columns={"fair_f1_fixed": "f1_cnn"})
    df_t = data["ts2vec_humidity"][["subject", "f1_score"]].rename(columns={"f1_score": "f1_ts2vec"})
    df = df_c.merge(df_t, on="subject")
    
    fig, ax = plt.subplots(figsize=(8, 6))
    x = np.arange(len(df))
    width = 0.35
    
    df = df.sort_values("f1_ts2vec", ascending=True)
    bars1 = ax.bar(x - width/2, df["f1_cnn"], width, label="CNN-LSTM (Supervised)", color=COLORS["compare"], edgecolor="black")
    bars2 = ax.bar(x + width/2, df["f1_ts2vec"], width, label="TS2Vec+Humidity (Self-supervised)", color=COLORS["ts2vec_humidity"], edgecolor="black")
    
    subj_labels = [s[:18] + "..." if len(s) > 18 else s for s in df["subject"]]
    ax.set_xticks(x)
    ax.set_xticklabels(subj_labels, rotation=45, ha="right", fontsize=8)
    ax.set_ylabel("F1 Score", fontsize=11)
    ax.set_title("Supervised vs Self-Supervised: CNN-LSTM vs TS2Vec+Humidity (18 subjects)", fontsize=12, fontweight="bold")
    ax.legend(loc="upper right")
    ax.set_ylim(0, 1.05)
    ax.grid(axis="y", alpha=0.3)
    
    mean_cnn = df["f1_cnn"].mean()
    mean_ts2 = df["f1_ts2vec"].mean()
    ax.text(0.98, 0.02, f"Mean: CNN-LSTM={mean_cnn:.3f}, TS2Vec+Hum={mean_ts2:.3f}", transform=ax.transAxes, ha="right", va="bottom", fontsize=10, bbox=dict(boxstyle="round", facecolor="wheat", alpha=0.7))
    
    plt.tight_layout()
    fig.savefig(os.path.join(out_dir, "ml_supervised_vs_selfsupervised.png"), dpi=150, bbox_inches="tight", facecolor="white")
    fig.savefig(os.path.join(out_dir, "ml_supervised_vs_selfsupervised.pdf"), bbox_inches="tight", facecolor="white")
    plt.close()
    print("Saved ml_supervised_vs_selfsupervised.png/pdf")

In [ ]:
def fig5_summary_table(metrics_df, out_dir):
    fig, ax = plt.subplots(figsize=(12, 4.5))
    ax.axis("off")
    
    table_data = []
    table_data.append(["Model", "Approach", "N", "F1\n(fixed)", "F1\n(val τ)", "Acc\n(fixed)", "Acc\n(val τ)"])
    for _, row in metrics_df.iterrows():
        name_short = MODEL_NAMES[row["model"]].replace("\n", " ")
        approach = "Supervised" if "compare" in row["model"] else "Self-supervised"
        table_data.append([
            name_short,
            approach,
            str(int(row["n_subjects"])),
            f"{row['f1_fixed']:.3f}",
            f"{row['f1_val_threshold']:.3f}",
            f"{row['acc_fixed']:.3f}",
            f"{row['acc_val_threshold']:.3f}",
        ])
    
    col_widths = [0.22, 0.14, 0.08, 0.14, 0.14, 0.14, 0.14]
    table = ax.table(cellText=table_data, loc="center", cellLoc="center", colWidths=col_widths)
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 2.2)
    
    for j in range(7):
        table[(0, j)].set_facecolor("#4A90D9")
        table[(0, j)].set_text_props(color="white", fontweight="bold")
    
    ax.set_title("Summary of Model Performance (LOSO Cross-Validation)", fontsize=12, fontweight="bold", pad=20)
    fig.savefig(os.path.join(out_dir, "ml_summary_table.png"), dpi=150, bbox_inches="tight", facecolor="white")
    fig.savefig(os.path.join(out_dir, "ml_summary_table.pdf"), bbox_inches="tight", facecolor="white")
    plt.close()
    print("Saved ml_summary_table.png/pdf")

In [ ]:
def main():
    os.makedirs(FIG_DIR, exist_ok=True)
    data = load_data()
    if not data:
        print("No result files found. Check paths.")
        return
    
    metrics_df = get_metrics(data)
    print("\nMean metrics:")
    print(metrics_df[["model", "f1", "accuracy", "n_subjects"]].to_string(index=False))
    
    fig1_model_comparison_bar(metrics_df, FIG_DIR)
    fig2_f1_boxplot(data, FIG_DIR)
    fig3_humidity_ablation(data, FIG_DIR)
    fig4_supervised_vs_selfsupervised(data, FIG_DIR)
    fig5_summary_table(metrics_df, FIG_DIR)
    
    print(f"\nAll save_model_mlp_xgb saved to {FIG_DIR}")

if __name__ == "__main__":
    main()



Mean metrics:
            model       f1  accuracy  n_subjects
          compare 0.474486  0.967090          20
  compare_swapped 0.983482  0.968118          20
  ts2vec_humidity 0.652103  0.954554          18
ts2vec_nohumidity 0.556351  0.936937          18
Saved ml_model_comparison.png/pdf
Saved ml_f1_boxplot.png/pdf
Saved ml_humidity_ablation.png/pdf
Saved ml_supervised_vs_selfsupervised.png/pdf
Saved ml_summary_table.png/pdf

All save_model_mlp_xgb saved to /Users/sonalimanoharan/Desktop/scientific_research/hw/save_model_mlp_xgb
